In [1]:
from datasets import load_dataset

dataset = load_dataset("openbmb/UltraFeedback")
data = [x for x in dataset['train']]

import json
from itertools import combinations
import random
random.seed(42)
new_data = []
for i, x in enumerate(data):
    # randomly sample 2 pairs
    all_combs = list(combinations(x['completions'], 2))
    random.shuffle(all_combs)
    
    for completion1, completion2 in all_combs[:2]:
        text1 = completion1['response']
        text2 = completion2['response']
        model1 = completion1['model']
        model2 = completion2['model']
        cand1_scores = {
            "human_preference": completion1['overall_score'],
        }
        cand1_scores.update({
            aspect: annot['Rating'] for aspect, annot in completion1['annotations'].items()
        })
        cand2_scores = {
            "human_preference": completion2['overall_score'],
        }
        cand2_scores.update({
            aspect: annot['Rating'] for aspect, annot in completion2['annotations'].items()
        })
        new_item = {
            "id": "ultrafeedback_{}_{}".format(x['source'], i),
            "instruction": x['instruction'],
            "input": "",
            "candidates": [
                {
                    "text": text1,
                    "model": model1,
                    "decoding_method": "unknown",
                    "scores": cand1_scores
                },
                {
                    "text": text2,
                    "model": model2,
                    "decoding_method": "unknown",
                    "scores": cand2_scores
                }
            ]
        }
        new_data.append(new_item)

print(len(new_data))
train_data = new_data[:-5000]
val_data = new_data[-5000:]
with open("train_data_prepared.json", "w") as f:
    json.dump(train_data, f, indent=4, ensure_ascii=False)
with open("val_data_prepared.json", "w") as f:
    json.dump(val_data, f, indent=4, ensure_ascii=False)

/home/dongfu/miniconda3/envs/llm-blender/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 448.54it/s]
Generating train split: 63967 examples [00:04, 15054.25 examples/s]


127932
